### Clean the data

imports

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv


## Connexion a la database

In [2]:
load_dotenv()
database_url = os.getenv("DATABASE_URL")
engine = create_engine(database_url)
print("Connexion réussie")

Connexion réussie


## Dossier contenant mes fichiers csv

In [3]:
csv_path = "csv/csv_used/dpe03existant.csv"
csv_folder = "csv/csv_used"

In [4]:
df_preview = pd.read_csv(csv_path, sep=',', quotechar='"', encoding='ISO-8859-1', nrows=5)
print(df_preview)
print(len(df_preview.columns))

      numero_dpe date_derniere_modification_dpe date_visite_diagnostiqueur  \
0  2229E0488205H                     2024-12-30                 2022-03-08   
1  2247E1057848I                     2024-12-30                 2022-05-11   
2  2275E0252200Y                     2024-06-25                 2022-02-02   
3  2233E0287314W                     2024-12-12                 2022-02-10   
4  2269E0225484E                     2024-12-30                 2022-02-03   

  date_etablissement_dpe date_reception_dpe date_fin_validite_dpe  \
0             2022-03-09         2022-03-10            2032-03-08   
1             2022-05-15         2022-05-16            2032-05-14   
2             2022-02-07         2022-02-08            2032-02-06   
3             2022-02-11         2022-02-14            2032-02-10   
4             2022-02-03         2022-02-04            2032-02-02   

   numero_dpe_remplace  numero_dpe_immeuble_associe        id_rnb  \
0                  NaN                         

In [8]:
from sqlalchemy import String
import ast

def parse_list_string(val):
    try:
        if pd.notnull(val):
            val = val.replace('\\/', '/')  # Évite les erreurs d'échappement invalides
            return ast.literal_eval(val)
        return None
    except Exception as e:
        print(f"Erreur de parsing pour : {val} -> {e}")
        return None



for file in os.listdir(csv_folder):
    if file.endswith(".csv") and file == "dpe03existant.csv":
        table_name = "table1"
        print(table_name)
        csv_path = os.path.join(csv_folder, file)
        print(f"Importing {file} into table {table_name}...")
        try:
            ad = pd.read_csv(csv_path, sep=',', nrows=10000, low_memory=False)
            ad.to_sql(table_name, engine, if_exists="append", index=False)
        except Exception as e:
            print(f"Erreur pour {file} : {e}")


table1
Importing dpe03existant.csv into table table1...


In [9]:
for file in os.listdir(csv_folder):
    if file.endswith(".csv") and file == "dpe03existant.csv":
        table_name = "tablefull"
        print(table_name)
        csv_path = os.path.join(csv_folder, file)
        print(f"Importing {file} into table {table_name}...")
        try:
            for chunk in pd.read_csv(csv_path, sep=',', chunksize=50000, low_memory=False):
                chunk.to_sql(table_name, engine, if_exists="append", index=False)
        except Exception as e:
            print(f"Erreur pour {file} : {e}")


tablefull
Importing dpe03existant.csv into table tablefull...
Erreur pour dpe03existant.csv : (psycopg2.errors.DiskFull) could not extend file "base/16555/26434159.15": No space left on device
HINT:  Check free disk space.

[SQL: INSERT INTO tablefull (numero_dpe, date_derniere_modification_dpe, date_visite_diagnostiqueur, date_etablissement_dpe, date_reception_dpe, date_fin_validite_dpe, numero_dpe_remplace, numero_dpe_immeuble_associe, id_rnb, numero_rpls_logement, numero_i ... 1208926 characters truncated ... dule__144)s, %(production_electricite_pv_kwhep_par_an__144)s, %(electricite_pv_autoconsommee__144)s)]
[parameters: {'apport_solaire_saison_chauffe__0': 842.5, 'facteur_couverture_solaire_saisi_installation_chauffage_n1__0': None, 'conso_refroidissement_ef__0': 0.0, 'facteur_couverture_solaire_installation_chauffage_n1__0': None, 'facteur_couverture_solaire_n1__0': None, 'surface_chauffee_installation_chauffage_n1__0': 42.0, 'emission_ges_chauffage_energie_n3__0': None, 'conso_c